## Regresja liniowa

In [ ]:
# On Unix
!wget https://raw.githubusercontent.com/jgrynczewski/datascience_demo/main/weights_heights.csv

In [ ]:
# On Windows
# !pip install wget
# !python -m wget https://raw.githubusercontent.com/jgrynczewski/datascience_demo/main/weights_heights.csv

# 1. Popatrzmy na dane

In [ ]:
# Import potrzebnych bibliotek

import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('weights_heights.csv', index_col='Index')
data.head(10)

In [ ]:
data.describe()

In [ ]:
# Wysokość przeliczamy na metry
data["Height"] = data["Height"] * 0.0254

In [ ]:
# Wagę przeliczamy na kilogramy
data["Weight"] = data["Weight"] * 0.4536

In [ ]:
# Wyświetlamy jeszcze raz data
data.head(10)

In [ ]:
data[["Height", "Weight"]].corr()

In [ ]:
data.plot(y="Height", color="red", title="Height distribution")

In [ ]:
data.plot(y="Weight", color="green", title="Weight distribution")

In [ ]:
data.plot(y="Height", kind="hist", color="red", title="Height distribution")

In [ ]:
data.plot(y="Weight", kind='hist', color="green", title="Weight distribution")

In [ ]:
def calc_bmi(height, weight):
  return weight / (height**2)

In [ ]:
data.plot(y='Height', x='Weight', kind='scatter', title='Height vs Weight')

Minimalizacja funkcji błędu

In [ ]:
# Przygotowanie danych pod minimalizację
# mnożenie macierzowe jako uogólnienie iloczynu skalarnego.

X = np.array([(x, 1) for x in data["Weight"]])  # dlaczego tak? mnożenie macierzowe
# -> y = w1*x + w0  <=> Y = w * X, gdzie w = [w1, w0], a X = [(x0, 1), (x1, 1), ...]
Y = data['Height']
print(X)
print(Y)

In [ ]:
# funkcja błędu

def error(w, X, Y):
  err = Y - np.matmul(X, w)
  return err.dot(err)

In [ ]:
# minimalizacja

# początkowy strzał
x0 = np.array([0, 0])

# scipy.optimize.minimize - minimization of scalar function of one or more variables

# parameters:
# fun: callable, the objective function to be minimized
# x0: ndarray, shape(n,) - initial guess  - that's how the minimize function
#                                           deduce the dimiension of the first
#                                           parameter of the fun.
# args: tuple (optional) - extra arguments passed to the objective function
#                          and its derivatives
# method str or callable - type od solver. If not given, chosen to be one
#                          of BFGS, L-BFGS-B, SLSQP, depending on whether
#                          or not the problem has constraints or bounds.
# bounds - sequence or Bounce (optional)

# returns:
# res: OptimizeResult
# The optimization result represented as a OptimizeResult object. Important
# attributes are: x the solution array, success a Boolean flag indicating
# if the optimizer exited successfully
result = scipy.optimize.minimize(
    error,
    x0=x0,
    args=(X, Y),
    method='L-BFGS-B',
    bounds=((-100, 100), (-10,10))
)

In [ ]:
w1, w0 = result.x
print(w1)
print(w0)

In [ ]:
x = np.linspace(min(data['Weight']), max(data['Weight']), 100)
y = w0 + x * w1   # hipoteza: korelacje pomiędzy wartościami height i weight można opisać funkcją liniową (regresja liniowa)

data.plot(
    y='Height',
    x='Weight',
    kind='scatter'
)

plt.plot(x, y, '-r')
plt.show()

In [ ]:
# Przewidywanie

x = 80
y = w0 + x*w1
print(y)